In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd
import evaluate
import numpy as np


In [6]:
def clean_dataset(filename, small_segment=True):
    dataset = pd.read_csv(filename)
    dataset = dataset[["segment_50","Class"]]
    dataset = dataset.rename(columns = {"segment_50":"text","Class":"label"})
    return dataset

dataset = clean_dataset('training_data_segmented.csv')
dataset.head()

,text,label
0,"[' shot in the line of duty; Gus Smith, 21, sh...",1
1,"['lk. valued at Ono, by a band of gunmen, wble...",0
2,[' sniper kills boy PHILADELPHIA - (AP) Hundre...,0
3,['ii Staff Writer The president of Michigan\'s...,1
4,"[' in signal, transportation, maintenance, mil...",0


In [7]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased2')
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased2')

In [8]:
from datasets import Dataset

dataset = Dataset.from_pandas(dataset)


def tokenize(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

dataset = dataset.map(tokenize, batched=True)

Map:   0%|          | 0/141 [00:00<?, ? examples/s]

In [9]:
mds = dataset.train_test_split(test_size = .2, shuffle=True, seed=42)

In [10]:
metric = evaluate.load('accuracy.py')

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [11]:
from transformers import TrainingArguments, Trainer

train_args = TrainingArguments(
    output_dir = "police_classifier",
    eval_strategy = "epoch",
    push_to_hub = False)


trainer = Trainer(
    model=model,
    args = train_args,
    train_dataset = ds['train'],
    eval_dataset = ds['test'],
    compute_metrics = compute_metrics)

trainer.train()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.565018,0.793103
2,No log,0.547789,0.724138
3,No log,0.510493,0.724138


TrainOutput(global_step=42, training_loss=0.5169201351347423, metrics={'train_runtime': 959.2916, 'train_samples_per_second': 0.35, 'train_steps_per_second': 0.044, 'total_flos': 88405314600960.0, 'train_loss': 0.5169201351347423, 'epoch': 3.0})

## Test Data